## 混合混触危険性予測テスト（2024.2作成）

畠山先生のアルゴリズムをもとに、混合混触危険性の予測に用いた最初のファイルです。この時点では理由生成をGPT-3.5に全て任せていました。また、予測結果は3値（Compatible, Caution, Incompatible）としていました。

In [ ]:
# 一通り動かしてみる

#ライブラリの自動インポート
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import openai
from tqdm import tqdm
from src.utils.GPT import json_generate
from src.qra_gen import generate_reason_and_predict_mixture

In [2]:
#api_key
openai.api_key = "api_key"

In [3]:
#laod dataset as dict
csv_path="dataset/Updated_Cameo_reactivity_sorted.csv"
df=pd.read_csv(csv_path)
df = df.iloc[:,:3]
chemical_records=df.to_dict(orient = "records")
chemical_records

[{'substance1': 'CARBON, ACTIVATED',
  'substance2': 'FERRIC SULFATE',
  'result': 'Compatible'},
 {'substance1': 'ALUMINUM ALKYL HYDRIDES',
  'substance2': 'FERRIC NITRATE',
  'result': 'Incompatible'},
 {'substance1': 'CARBON, ACTIVATED',
  'substance2': 'METHANE',
  'result': 'Compatible'},
 {'substance1': 'AIR, COMPRESSED',
  'substance2': 'WATER',
  'result': 'Compatible'},
 {'substance1': 'ALUMINUM ALKYL HYDRIDES',
  'substance2': 'WATER',
  'result': 'Incompatible'},
 {'substance1': 'FERRIC SULFATE',
  'substance2': 'IRON OXIDE, SPENT',
  'result': 'Incompatible'},
 {'substance1': 'FERRIC NITRATE',
  'substance2': 'IRON, [POWDERED]',
  'result': 'Incompatible'},
 {'substance1': 'ETHANOL',
  'substance2': 'PROPIONALDEHYDE',
  'result': 'Caution'},
 {'substance1': 'CARBON, ACTIVATED',
  'substance2': 'FERRIC CHLORIDE',
  'result': 'Compatible'},
 {'substance1': 'IRON OXIDE, SPENT',
  'substance2': 'WATER',
  'result': 'Incompatible'},
 {'substance1': 'PROPIONALDEHYDE',
  'substanc

In [4]:
#このプロンプトを変更

# 質問文を生成するための指示
qa_gen_command = """
Provide the quantitative reasons within 300 words so that a scientist, who does not know the compatibility or reactivity of a mixture, can predict these outcomes.
We must quantitatively consider how the combination of substances influences their compatibility or reactivity.
I absolutely forbid you to make qualitative generalizations.

#Bad example reasons
## Its molecular weight compared to simpler compounds, contributing to higher reactivity. (qualitative discussion is practically meaningless!!!)
## Therefore, the mixture is highly reactive. (Never include the answer in the reason!!).

#Good example reasons
## Mixing sodium with water generates a significant amount of heat and hydrogen gas due to the chemical reaction, which can lead to an explosive reaction under certain conditions.
## The combination of acetone and bleach creates chloroform and other hazardous compounds due to the chemical reaction between acetone's methyl groups and sodium hypochlorite in bleach.

#Output: Reason key
"""

# 分子構造から物性を予測するための指示
#predict_command = """
#Predict the compatibility or reactivity of the following mixture.
#Consider the quantitative reasons provided and, in any case, output a prediction about the mixture's safety or potential hazards.

#Output: Result key
#"""

predict_command = """
Predict the compatibility or reactivity of the following mixture, using only one of the following outcomes: Compatible, Incompatible, Caution. 
Consider the quantitative reasons provided and make your prediction based on the mixture's safety or potential hazards.

#Output: Result key should be one of [Compatible, Incompatible, Caution].
"""


#use GPT3.5 completion
llm_ask_func=json_generate

#if gen_reason=True, generate reason and predict
#if gen_reason=False, only predict without reason (for control)
#gen_reason=True
gen_reason=False

#n回のpredictionを行う
n_trials=3

In [7]:


completed_records = []
for record in tqdm(chemical_records):  # tqdmは進捗状況を表示
    try:
        print("Hello")
            # generate_reason_and_predict関数により、理由生成と予測実行
        completed_record = generate_reason_and_predict_mixture(
            substance1=record["substance1"],
            substance2=record["substance2"],
            result=record["result"],  # resultは混合物の危険性判定の結果
            qa_gen_command=qa_gen_command,  # 理由を尋ねるための指示
            predict_command=predict_command,  # 予測を行うための指示
            llm_ask_func=json_generate,  # LLM関数
    #            smiles=record.get("smiles", ""),  # SMILES情報（存在する場合）
            gen_reason=gen_reason,  # 理由を生成するかどうか
            n_trials=n_trials  # 予測を試みる回数
        )
#        print(completed_record)
    except Exception as e:
        print("negative")
#        traceback.print_exc()
        print(e)  # 例外が発生した場合にエラーを表示
        completed_record = record  # 例外が発生した場合は元のレコードを使用

    completed_records.append(completed_record)  # 処理されたレコードをリストに追加


  0%|                                                                                           | 0/91 [00:00<?, ?it/s]

Hello
{'Result': 'Incompatible'}
{'Result': 'Incompatible'}


  1%|▉                                                                                  | 1/91 [00:01<02:49,  1.89s/it]

{'Result': 'Incompatible'}
Hello
{'Result': 'Incompatible'}
{'Result': 'Incompatible'}


  2%|█▊                                                                                 | 2/91 [00:03<02:28,  1.67s/it]

...

{'Result': 'Incompatible'}
Hello
{'Result': 'Compatible'}
{'Result': 'Compatible'}


 99%|█████████████████████████████████████████████████████████████████████████████████ | 90/91 [02:21<00:02,  2.05s/it]

{'Result': 'Compatible'}
Hello
{'Result': 'Compatible'}
{'Result': 'Compatible'}


100%|██████████████████████████████████████████████████████████████████████████████████| 91/91 [02:23<00:00,  1.57s/it]

{'Result': 'Compatible'}


In [8]:
completed_records

[{'substance1': 'CARBON, ACTIVATED',
  'substance2': 'FERRIC SULFATE',
  'result': 'Compatible',
  'generated_reason': '',
  'predicted_results': ['Incompatible', 'Incompatible', 'Incompatible']},
 {'substance1': 'ALUMINUM ALKYL HYDRIDES',
  'substance2': 'FERRIC NITRATE',
  'result': 'Incompatible',
  'generated_reason': '',
  'predicted_results': ['Incompatible', 'Incompatible', 'Incompatible']},
 {'substance1': 'CARBON, ACTIVATED',
  'substance2': 'METHANE',
  'result': 'Compatible',
  'generated_reason': '',
  'predicted_results': ['Incompatible', 'Incompatible', 'Incompatible']},
 {'substance1': 'AIR, COMPRESSED',
  'substance2': 'WATER',
  'result': 'Compatible',
  'generated_reason': '',
  'predicted_results': ['Caution', 'Caution', 'Caution']},
 {'substance1': 'ALUMINUM ALKYL HYDRIDES',
  'substance2': 'WATER',
  'result': 'Incompatible',
  'generated_reason': '',
  'predicted_results': ['Incompatible', 'Incompatible', 'Incompatible']},
 {'substance1': 'FERRIC SULFATE',
  'subs

In [9]:
#save
import datetime
import json

study_name="mixture_practice_wo_reason"
now=datetime.datetime.now()
now_str=now.strftime("%Y%m%d_%H%M%S")
save_path=f"results/{study_name}_{now_str}.json"
with open(save_path, 'w', encoding = "utf-8") as f:
    json.dump(completed_records, f, indent=4, ensure_ascii=False)